## Automating Table Shortcuts in Microsoft Fabric for Unified Data Strategy in OneLake
**by Jesus Lopez Martin - https://www.syntax.es - November 2024**

This Python script demonstrates how to automate the creation of shortcuts for tables between a source Workspace and Lakehouse and a destination Workspace and Lakehouse in Microsoft Fabric. It aligns with the "single source of truth" strategy in OneLake and supports the Medallion architecture, simplifying data access and management across workspaces. By leveraging the Microsoft Fabric APIs and the semantic-link-labs library, this approach helps streamline data integration and reduce redundancy.

**Key Benefits:**
- Centralized Data Access: This script supports the "single source of truth" strategy by enabling seamless access to tables across workspaces and Lakehouses without data duplication.
- Simplified Management: Automates the creation of shortcuts, reducing manual effort and operational overhead.
- Medallion Architecture Support: Facilitates the creation of Bronze, Silver, and Gold layers in Microsoft Fabric by linking data across Lakehouses.


In [ ]:
import notebookutils
import sempy_labs as labs
from sempy import fabric
import requests
import json

# Retrieve access token for Microsoft Fabric API
access_token = notebookutils.credentials.getToken('https://api.fabric.microsoft.com/')

# Define source and destination workspace and lakehouse details
destination_workspace_id = fabric.resolve_workspace_id(workspace="TEST Workspace")
destination_lakehouse_id = labs.resolve_lakehouse_id(lakehouse="TEST_Destination_Lakehouse", workspace="TEST Workspace")
source_workspace_id = fabric.resolve_workspace_id(workspace="TEST Source Workspace")
source_lakehouse_id = labs.resolve_lakehouse_id(lakehouse="Source_Lakehouse", workspace="TEST Source Workspace")

# API URLs
list_tables_url = f'https://api.fabric.microsoft.com/v1/workspaces/{source_workspace_id}/lakehouses/{source_lakehouse_id}/tables'
create_shortcut_url = f'https://api.fabric.microsoft.com/v1/workspaces/{destination_workspace_id}/items/{destination_lakehouse_id}/shortcuts'

# Common headers for API requests
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Step 1: List tables in the source Lakehouse
response = requests.get(list_tables_url, headers=headers)

if response.status_code == 200:
    tables = response.json().get('data', [])
    print(f'Found {len(tables)} tables in the source Lakehouse.')

    # Step 2: Create shortcuts in the destination Lakehouse
    for table in tables:
        table_name = table['name']
        shortcut_data = {
            "path": "Tables",
            "name": table_name,
            "target": {
                "oneLake": {
                    "workspaceId": source_workspace_id,
                    "itemId": source_lakehouse_id,
                    "path": f"Tables/{table_name}"
                }
            }
        }
        response_shortcut = requests.post(create_shortcut_url, headers=headers, data=json.dumps(shortcut_data))
        if response_shortcut.status_code == 201:
            print(f'Successfully created shortcut for table "{table_name}".')
        else:
            print(f'Failed to create shortcut for table "{table_name}": {response_shortcut.status_code} - {response_shortcut.text}')
else:
    print(f'Failed to list tables in the source Lakehouse: {response.status_code} - {response.text}')
